In [97]:
!pip install soundfile
!pip install pandas
!pip install scipy
!pip install librosa
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from pathlib import Path
import pandas as pd
import numpy as np
import soundfile as sf
import scipy
from scipy import fft
import functools
from torch import nn
import librosa
import math
import torchaudio 
import math
import yaml
torch.__version__


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


'1.9.0+cu102'

In [98]:
# data_dir should be structured as follows:
# - speaker_id_1/
# - speaker_id_1/sample_id1.wav
# - speaker_id_1/sample_id1.csv
# - speaker_id_1/sample_id2.wav
# - speaker_id_1/sample_id2.wav
# - speaker_id_2/sample_id1.wav
# ..
class VisemeDataset(Dataset):
    def __init__(self, data_dir, audio_transform, viseme_transform):
        self.viseme_transform= viseme_transform
        self.audio_transform = audio_transform
        self.audio_files = []
        self.visemes = []
        self.processed = {}
        for file in list(Path(data_dir).rglob("*.wav")):
            self.audio_files.append(file)
            self.visemes.append(str(file).replace("wav", "csv"))

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        if idx not in self.processed:
            fft, num_samples, mask = self.audio_transform(self.audio_files[idx])
            fft = torch.tensor(fft, dtype=torch.float)
            viseme_filename = self.visemes[idx]
            visemes = torch.tensor(self.viseme_transform(viseme_filename).values.astype(np.float32))       
            #assert(visemes.shape[0] == fft.shape[0])
            self.processed[idx] =fft, torch.tensor(mask), visemes, viseme_filename
        return self.processed[idx]

class SeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bias=False):
        super(SeparableConv1d, self).__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size=kernel_size, 
                               groups=in_channels, bias=bias, padding=1)
        self.pointwise = nn.Conv1d(in_channels, out_channels, 
                               kernel_size=1, bias=bias)
    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out
    
class Conv1dModel(nn.Module):
    def __init__(self, seq_length=633, n_ffts=2457, num_viseme=4, ks=256):
        super(Conv1dModel, self).__init__()
        self.conv = nn.Sequential(
            SeparableConv1d(seq_length,seq_length,ks),
            nn.ReLU(),
            SeparableConv1d(seq_length,seq_length,ks),
            nn.ReLU(),
        )
        self.attention = nn.MultiheadAttention(1951, 1, batch_first=True)
        #self.linear_relu_stack2 = nn.Sequential(
        #    SeparableConv1d(seq_length,seq_length,ks),
        #    #nn.Linear(seq_length, seq_length),
        #    nn.ReLU(),7
        #)
        self.linear_out = nn.Linear(1951, 11)

    def forward(self, x):
        o1 = self.conv(x)
        attn_output, attn_output_weights = self.attention(o1, o1, o1)
        attn_output = attn_output.tile((4,1,1,1)).transpose(0,1)
        o1 = self.linear_out(attn_output)        
        return o1

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim=None, hidden_size=512, num_viseme=4):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_dim, hidden_size, 1, bidirectional=True)
        self.attention = nn.MultiheadAttention(hidden_size*2, 1, batch_first=True)       
        self.l1 = torch.nn.Linear(hidden_size*2, 1)
        self.l2 = torch.nn.Linear(hidden_size*2, 1)
        self.l3 = torch.nn.Linear(hidden_size*2, 1)
        self.l4 = torch.nn.Linear(hidden_size*2, 1)

    def forward(self, x):
        out_f, _ = self.lstm(x)
        #out_f = out_f[:,:,:self.embed_dim] + out_f[:,:,self.embed_dim:]
        attn_output, attn_output_weights = self.attention(out_f, out_f, out_f)
        return torch.stack([
            self.l1(attn_output),
            self.l2(attn_output),
            self.l3(attn_output),
            self.l4(attn_output)
        ],dim=1)




In [99]:
def preprocess_viseme(csv, pad_len_in_secs=None, target_framerate=None, blendshapes=None):
    csv = pd.read_csv(csv)
    
    # first, drop every nth row to reduce effective framerate    
    csv = csv.iloc[::int(59.97 / target_framerate)]
    
    pad_len = int(pad_len_in_secs * target_framerate)
        
    if(csv.shape[0] < pad_len):
        pad = pd.DataFrame(0, index=[i for i in range(pad_len - csv.shape[0])], columns=csv.columns)
        pad.pad(inplace=True)
        csv = pd.concat([csv, pad])
    else:
        csv = csv.iloc[:pad_len]
        #print("Visemes exceeded max length, truncate?")
    columns = list(csv.columns)
    columns.remove("Timecode")
    
    return csv[blendshapes] if blendshapes is not None else csv



In [100]:
def pad_audio(audio, sample_rate, pad_len_in_secs):
    # left-pad the audio so we have the left context when starting at the initial viseme
    pad_len_in_samples = pad_len_in_secs * sample_rate 
    if len(audio.shape) > 1:
        audio = audio[0]
    if audio.shape[0] < pad_len_in_samples:
        audio = np.pad(audio, (0, (pad_len_in_secs * sample_rate) - audio.shape[0]), constant_values=0.001)
    elif audio.shape[0] > pad_len_in_samples:
        audio = audio[:pad_len_in_samples]
    #audio = np.hstack([np.zeros((win_length*2,)), audio])
    return audio

def load_and_pad_audio(filepath, resample_to, pad_len_in_secs):
    audio, rate = sf.read(filepath)
    audio = librosa.resample(audio, rate, resample_to)
    audio = pad_audio(audio, resample_to, pad_len_in_secs)
    return audio

\---S1---/\---S2---/\---S3---/
WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
____________\---V1---/
_____________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

             \---S1---/\---S2---/\---S3---/
             WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
______________________\---V2---/
_____________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


A = audio sample
V1, V2, etc = viseme 1, viseme 2, etc (aka "frame", but not in the MFCC sense)
W = a window of audio samples that will be used as input (aka "frame" in the MFCC sense. to avoid confusion, we refer to this as the "window" and the viseme as the "frame")
S1, S2, S3 = STFT of a window 
num_frames == num_windows

audio_bins_per_window = the number of S per W
samples_per_bin = the length of each S (i.e. number of As)
bin_hop_length = the number of A between S1 and S2 (in the picture above, hop_length == len(S1) == len(S2) == samples_per_bin

in practice, we calculate as such:
          /-----------V2--------------\
/--------------V1------------\
\---S1---/\---S2---/\---S3---/\---S4---/\---S5---/\---S6---/
_______________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

then at V1, we take {S1,S2,S3}, at V2 we take {S4,S5,S6}, etc

# STFT hop length should equal to length of a single viseme frame, in samples

In [101]:
def stft(filepath, 
         frame_len, 
         window_len, 
         stft_frames_per_window,
         resample_to, 
         pad_len_in_secs, 
         n_mels):
    
    audio = load_and_pad_audio(filepath, pad_len_in_secs, resample_to)
    #wdw = np.hanning(audio_window_in_samples)
    
    num_frames = audio.shape[0] // frame_len
    # TODO - mask for padding 
    # [1] * actual_seq_length + [0] * (padded_seq_length - actual_seq_length)
    
    # take the STFT of the entire audio file 
    # with a window size equivalent to the audio_window_in_samples / audio_bins_per_window
    n_fft = int(window_len / stft_frames_per_window)

    transformed = librosa.stft(audio, 
                               n_fft=n_fft,
                               win_length=n_fft,
                               hop_length=n_fft)
    
    melfb = librosa.filters.mel(resample_to, n_fft, n_mels=n_mels)    
    mels = np.dot(melfb, np.abs(transformed))
    
    log_mels = np.log(mels)
    
    output = mels_to_mfccs(log_mels, padded_seq_length, stft_frames_per_window, n_mels)

    return output, audio.shape[0], 



In [112]:
# let num_stft_frames be the number of STFT frames per audio sample 
# num_stft_frames == (audio_len / n_fft)
# (assuming FFT window length == hop length == n_ffts)
# coeffs will be B x num_stft_frames x num_mels
# every 
# if we've calculated the FFT size correctly, the hop length will be half 
# so stft_frames_per_window should be odd

def coeffs_to_windows(coeffs, num_frames, stft_frames_per_window, n_mels):
    
    output = np.zeros((num_frames, stft_frames_per_window*n_mels))
    
    # the number of frames to hop is just half the number of STFT frames per window
    hop_len = int(stft_frames_per_window / 2)
    
    for i in range(num_frames):
        c = coeffs[:, (i*hop_len):(i*hop_len)+stft_frames_per_window]
        # padding
        if(c.shape[1] < stft_frames_per_window):
            c = np.pad(c, [(0,0), (0, stft_frames_per_window - c.shape[1])], constant_values=0)
        
        output[i, :] = np.reshape(c, stft_frames_per_window*n_mels)
    return output


# featurize audio in exactly the same way as TensorflowTTS
# this enables mels from the synthesis step to be reused for viseme prediction

def tftts_mels(filepath, 
               config=None):
    
    audio = load_and_pad_audio(filepath, config["sampleRate"], config["paddedAudioLength"])
     
    # this is (mostly) copied verbatim from https://github.com/TensorSpeech/TensorFlowTTS/blob/master/tensorflow_tts/bin/preprocess.py @ 4a7d584 
    # except the hop length must be the size of the window (in samples) divided by the number of STFT frames per window
    assert(config["hopSize"] == int(config["windowLength"] // config["stftFramesPerWindow"]))
    
    D = librosa.stft(
        audio,
        n_fft=config["fftSize"],
        hop_length=config["hopSize"],
        win_length=config["fftSize"],
        #config["windowLength"] if "windowLength" in config else config["fftSize"],
        window=config["window"],
        pad_mode="reflect",
    )
    
    S, _ = librosa.magphase(D)  # (#bins, #frames)
    fmin = 0 if config["fmin"] is None else config["fmin"]
    fmax = sampling_rate // 2 if config["fmax"] is None else config["fmax"]
    mel_basis = librosa.filters.mel(
        sr=config["sampleRate"],
        n_fft=config["fftSize"],
        n_mels=config["numMels"],
        fmin=fmin,
        fmax=fmax,
    )
    log_mels = np.log10(np.maximum(np.dot(mel_basis, S), 1e-10)).T  # (#frames, #bins)
    
    # coeffs = fft.dct(log_mels.T) <-- where did this come from?
    
    coeffs = log_mels.T
    
    num_frames = (config["sampleRate"] * config["paddedAudioLength"]) // config["frameLength"]

    output = coeffs_to_windows(coeffs, num_frames,  config["stftFramesPerWindow"], config["numMels"])

    return output, audio.shape[0], [1] * log_mels.shape[0]


In [113]:
from tensorflow_tts.inference import AutoConfig
import json

from collections import OrderedDict

mappings = OrderedDict()
mappings["MouthClose"] = "Mouth_Lips_Open"
mappings["MouthFunnel"] = "A29_Mouth_Funnel"
mappings["MouthPucker"] = "A30_Mouth_Pucker"
mappings["JawOpen"] = "Mouth_Open"
print(list(mappings.keys()))
# config for the input blendshapes
source_config = {
    # source framerate for raw viseme label input
    "framerate":59.97,
    # we need to map the blendshapes from the incoming CSV from LiveLinkFace to CC3 blendshapes
    # unfortunately not 1-to-1, but seems to work well enough
    "mappings":mappings
}
 
# config for the viseme model
model_config = {}
# the filename for the trained model that we will export below
model_config["modelPath"] = "bilstm.tflite"
# the blendshapes that will be predicted
# we also need to export this so the gltf animator can match the model outputs indices to morph target indices
model_config["targetNames"] = ["Mouth_Lips_Open", "A29_Mouth_Funnel", "A30_Mouth_Pucker", "Mouth_Open"]

# actual framerate to use for viseme labels. 
# raw labels will be resampled/transformed (either averaged or simply dropped).
model_config["frameRate"] = source_config["framerate"] / 2
# the sample rate that audio will be resampled to
model_config["sampleRate"] = 22050

# the duration of a viseme frame is (1 / target_framerate) seconds
model_config["frameLength"] = math.ceil(model_config["sampleRate"] * (1 / model_config["frameRate"]))
# all audio will be padded to the following size
model_config["paddedAudioLength"] = 10
# the raw input for each viseme frame will be an audio window of size X 
# the middle sample of the viseme frame is aligned with the middle sample of the audio window
# this means, at the nominal "anchor sample" of the viseme frame, there will be 
# X/2 samples to the left and X/2 samples to the right
# let's use 0.5 seconds
model_config["windowLength"] = int(0.5 * model_config["sampleRate"]) 

# this raw audio input will then be transformed into a number of STFT frames/coefficients
# each viseme frame will have this number of STFT frames, which will be the actual input at each timestep
# Since audio windows overlap, we won't want to waste cycles repeatedly computing the STFT across the whole audio sequence
# So in practice, we pre-calculate the STFT for the whole sequence, then just sub-sample the coefficients at each timestep
# when assigning STFT frames, the hop length will then just be half this value
model_config["stftFramesPerWindow"] = 33

model_config["hopSize"] = int(model_config["windowLength"] // model_config["stftFramesPerWindow"])

# load the TTS config so we can match the same parameters 
# this may override some of the parameters set above
USE_TFTTS_CONFIG=True
if USE_TFTTS_CONFIG:
    with open("/mnt/hdd_2tb/home/hydroxide/projects/TensorFlowTTS/preprocess/baker_preprocess.yaml", "r") as f:
        tftts_config = yaml.safe_load(f)
        model_config["fftSize"] = tftts_config["fft_size"]
        model_config["numMels"] = tftts_config["num_mels"]       
        if "window_length" in tftts_config:
            assert(tftts_config["window_length"] == model_config["windowLength"])
        
        model_config["window"] = tftts_config["window"]
        model_config["fmin"] = tftts_config["fmin"]
        model_config["fmax"] = tftts_config["fmax"]
        assert(tftts_config["sampling_rate"] == model_config["sampleRate"])
        
        #config={"num_mels":80, "sampling_rate":22050,"fmin":80,"fmax":6000, "window":"hann", "fft_size":512, "hop_size":hop_size})

#seq_length = math.ceil((pad_len_in_secs * resample_to) / frame_len)

# save the config so we can pass
with open("viseme_model.json", "w",encoding="utf-8") as outfile:
    print("Dumping")
    json.dump(model_config, outfile)
    
process_audio = None

if USE_TFTTS_CONFIG:
    process_audio = functools.partial(tftts_mels, 
                                          config=model_config)
else:
    num_mels=39
    process_audio = functools.partial(stft, model_config);
        #viseme_frame_len_in_samples=viseme_frame_len_in_samples, # this refers to the size of the viseme/audio window,
        #audio_window_in_samples=audio_window_in_samples, # TODO - update these
        #stft_frames_per_window=stft_frames_per_window,
        #resample_to=resample_to, 
        #pad_len_in_secs=pad_len_in_secs,
        #n_mels=num_mels)

['MouthClose', 'MouthFunnel', 'MouthPucker', 'JawOpen']
Dumping


In [114]:
batch_size = 20
    
process_viseme = functools.partial(preprocess_viseme, 
                                   pad_len_in_secs=model_config["paddedAudioLength"], 
                                   blendshapes=source_config["mappings"], 
                                   target_framerate=model_config["frameRate"])

training_data = VisemeDataset("./data/training/speaker_1/", 
                              process_audio, \
                              process_viseme)
test_data = VisemeDataset("./data/test/speaker_1/", 
                              process_audio, \
                              process_viseme)
def collate_samples(feat_tuples):
    return feat_tuples
    padded = torch.nn.utils.rnn.pad_sequence([f[0] for f in feat_tuples], batch_first=True, padding_value=0.0)
    #mask = torch.stack([feat_tuples[i][1] for i in range(len(feat_tuples))])
    labels = torch.nn.utils.rnn.pad_sequence([f[2] for f in feat_tuples], batch_first=True, padding_value=0.0)
    viseme_filenames = [feat_tuples[i][3] for i in range(len(feat_tuples))]
    
    return padded, labels,viseme_filenames
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

model_config

Using cuda device


{'modelPath': 'bilstm.tflite',
 'targetNames': ['Mouth_Lips_Open',
  'A29_Mouth_Funnel',
  'A30_Mouth_Pucker',
  'Mouth_Open'],
 'frameRate': 29.985,
 'sampleRate': 22050,
 'frameLength': 736,
 'paddedAudioLength': 10,
 'windowLength': 11025,
 'stftFramesPerWindow': 33,
 'hopSize': 334,
 'fftSize': 512,
 'numMels': 80,
 'window': 'hann',
 'fmin': 80,
 'fmax': 7600}

In [ ]:
input_dim = int(model_config["windowLength"] / model_config["hopSize"] * model_config["numMels"])
#print(stft_frames_per_window)
#print(tftts_config["hop_size"])
print(input_dim)
model = BiLSTMModel(hidden_size=512, input_dim=input_dim).to(device)

learning_rate = 0.001
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
num_steps = 100000
print_loss_every = 50
eval_every = 500

batch = iter(train_dataloader)
accum_loss = 0

for t in range(num_steps):
    optimizer.zero_grad()
    train_features, train_mask, train_labels, train_files = next(batch, (None,None,None,None))
    #print(train_features.shape)
    #print(train_labels.shape)

    if train_features is None:
        batch = iter(train_dataloader)
        train_features, train_mask , train_labels, _ = next(batch)
    
    #train_mask = torch.unsqueeze(train_mask, 2)
    
    x = train_features.to(device)     
    
    y = train_labels.to(device) #* train_mask.to(device)
    
    preds = model(x) #* train_mask.to(device)
    
    #print(preds.shape)
    
    preds = torch.transpose(preds, 1,3).squeeze()
    #print(f"x {x.shape} preds {preds.shape} y {y.shape}")    
    #loss = torch.nn.functional.cross_entropy(preds, y)
    loss = torch.nn.functional.huber_loss(preds, y)
    if len(preds.shape) < 3:
        preds = torch.unsqueeze(preds, 0)
    #print(preds.shape)
    for i in range(preds.shape[1] - 1):
        loss += torch.nn.functional.cosine_embedding_loss(preds[:,i,:], preds[:,i+1,:], (torch.ones(preds.shape[0])).to(device))
    accum_loss += loss.item()
    if t % print_loss_every == 0:
        print(f"Step {t} Avg loss: {accum_loss / print_loss_every}")
        accum_loss = 0
    loss.backward()
    optimizer.step()
    
    
    
    if t > 0 and t % eval_every == 0:
        accum_loss = 0
        for test_features, test_mask, test_labels, _ in iter(test_dataloader):
            x = test_features.to(device)
            #x = torch.transpose(x, 1, 2)
            #x = torch.unsqueeze(x, dim=3)
            y = test_labels.to(device)
            preds = model(x)
            preds = torch.transpose(preds, 1,3)
            accum_loss += torch.nn.functional.mse_loss(preds, y).item()
            #accum_loss = torch.nn.functional.cross_entropy(preds, y)

        print(f"Test loss {accum_loss}")
        accum_loss = 0
    
#pred_probab = nn.Softmax(dim=1)(logits)
#y_pred = pred_probab.argmax(1)
#print(f"Predicted class: {y_pred}")

2640
torch.Size([20, 299, 4])
Step 0 Avg loss: 0.000343509241938591
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])


/home/hydroxide/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Using a target size (torch.Size([1, 299, 4])) that is different to the input size (torch.Size([299, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
Step 50 Avg loss: 3.736698604337871
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 

torch.Size([20, 299, 4])
Step 350 Avg loss: 0.008761207442730665
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([

/home/hydroxide/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Using a target size (torch.Size([20, 299, 4])) that is different to the input size (torch.Size([20, 1, 299, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/home/hydroxide/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Using a target size (torch.Size([2, 299, 4])) that is different to the input size (torch.Size([2, 1, 299, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Test loss 0.028751344420015812
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299,

torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
Step 850 Avg loss: 0.00834678984247148
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([

torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
Step 1150 Avg loss: 0.007972445520572365
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size

torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([1, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
torch.Size([20, 299, 4])
t

In [ ]:
#test_features, test_mask, test_labels, test_files = next(iter(test_dataloader))
#test_features, test_mask, test_labels, test_files = next(iter(train_dataloader))
##test_features = torch.transpose(test_features, 1, 2)
test_features = train_features
export_y_batch = model(test_features.to(device)) 
export_y = export_y_batch[0,:,:]
print(export_y.shape)
header = "Timecode,BlendShapeCount,eyeBlinkRight,eyeLookDownRight,eyeLookInRight,eyeLookOutRight,eyeLookUpRight,eyeSquintRight,eyeWideRight,eyeBlinkLeft,eyeLookDownLeft,eyeLookInLeft,eyeLookOutLeft,eyeLookUpLeft,eyeSquintLeft,eyeWideLeft,jawForward,jawRight,jawLeft,jawOpen,mouthClose,mouthFunnel,mouthPucker,mouthRight,mouthLeft,mouthSmileRight,mouthSmileLeft,mouthFrownRight,mouthFrownLeft,mouthDimpleRight,mouthDimpleLeft,mouthStretchRight,mouthStretchLeft,mouthRollLower,mouthRollUpper,mouthShrugLower,mouthShrugUpper,mouthPressRight,mouthPressLeft,mouthLowerDownRight,mouthLowerDownLeft,mouthUpperUpRight,mouthUpperUpLeft,browDownRight,browDownLeft,browInnerUp,browOuterUpRight,browOuterUpLeft,cheekPuff,cheekSquintRight,cheekSquintLeft,noseSneerRight,noseSneerLeft,tongueOut,HeadYaw,HeadPitch,HeadRoll,LeftEyeYaw,LeftEyePitch,LeftEyeRoll,RightEyeYaw,RightEyePitch,RightEyeRoll".split(',')
selected_output_indices = [header.index(x[0].lower() + x[1:]) for x in blendshapes]
num_visemes = len(blendshapes)
with open("output.csv", "w") as outfile:
    outfile.write(",".join(header) + "\n")
    timer_ms = 0
    print(export_y)
    for t in range(export_y.shape[1]):
        output = [str(0)] * len(header)
        second = str(int(timer_ms // 1000)).zfill(2)
        frame = (timer_ms % 1000) * target_framerate / 1000
        output[0] = f"00:00:{second}:{frame}"
        for viseme in range(num_visemes): 
            output[selected_output_indices[viseme]] = str(export_y[viseme,t,:].item())
        timer_ms += (1 / target_framerate) * 1000
        outfile.write(",".join(output) + "\n")

In [ ]:
train_files[0]

In [ ]:
new_header = "Timecode,BlendShapeCount,EyeBlinkLeft,EyeLookDownLeft,EyeLookInLeft,EyeLookOutLeft,EyeLookUpLeft,EyeSquintLeft,EyeWideLeft,EyeBlinkRight,EyeLookDownRight,EyeLookInRight,EyeLookOutRight,EyeLookUpRight,EyeSquintRight,EyeWideRight,JawForward,JawRight,JawLeft,JawOpen,MouthClose,MouthFunnel,MouthPucker,MouthRight,MouthLeft,MouthSmileLeft,MouthSmileRight,MouthFrownLeft,MouthFrownRight,MouthDimpleLeft,MouthDimpleRight,MouthStretchLeft,MouthStretchRight,MouthRollLower,MouthRollUpper,MouthShrugLower,MouthShrugUpper,MouthPressLeft,MouthPressRight,MouthLowerDownLeft,MouthLowerDownRight,MouthUpperUpLeft,MouthUpperUpRight,BrowDownLeft,BrowDownRight,BrowInnerUp,BrowOuterUpLeft,BrowOuterUpRight,CheekPuff,CheekSquintLeft,CheekSquintRight,NoseSneerLeft,NoseSneerRight,TongueOut,HeadYaw,HeadPitch,HeadRoll,LeftEyeYaw,LeftEyePitch,LeftEyeRoll,RightEyeYaw,RightEyePitch,RightEyeRoll"
remap = {h:(h[0].lower() + h[1:]) if h not in ["Timecode","BlendShapeCount","HeadYaw","HeadPitch","HeadRoll","LeftEyeYaw","LeftEyePitch","LeftEyeRoll","RightEyeYaw","RightEyePitch","RightEyeRoll"]  else h for h in new_header.split(",") }
for oh in remap.values():
    if oh not in header:
        print(oh)

def new_to_old(csv_df):
    return csv_df.rename(columns=remap)
df = preprocess_viseme("data/training/speaker_1/20210824_1/61.csv", pad_len_in_secs=pad_len_in_secs, 
                                   resample_to=target_framerate)
#print(df)
df = new_to_old(df)
cols = [c for c in df.columns if c not in ["jawOpen", "mouthClose", "Timecode"]]
df[cols] = 0
df = df[df["Timecode"] != 0]
df.to_csv("original.csv", index=False)
#new_to_old(df)[header].to_csv("original.csv", index=False)
df

In [ ]:
input_dim

In [ ]:
torch.save(model, "bilstm.torch")

In [ ]:
for item in ["MouthClose", "MouthFunnel", "MouthPucker", "JawOpen"]:
    print(new_header.split(",").index(item))

In [3]:
torch.load("bilstm.torch")

AttributeError: Can't get attribute 'BiLSTMModel' on <module '__main__'>

In [5]:
model = torch.load("bilstm.torch",map_location=torch.device('cpu'))
import torch.onnx 

# set the model to inference mode 
model.eval() 

# Let's create a dummy input tensor  
dummy_input = torch.randn(1, 119, 2640, requires_grad=False)  

# Export the model   
torch.onnx.export(model,         # model being run 
     dummy_input,       # model input (or a tuple for multiple inputs) 
     "bilstm.onnx",       # where to save the model  
     export_params=True,  # store the trained parameter weights inside the model file 
     opset_version=10,    # the ONNX version to export the model to 
     do_constant_folding=True,  # whether to execute constant folding for optimization 
     #input_names = ['modelInput'],   # the model's input names 
     #output_names = ['modelOutput'], # the model's output names 
     dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
    'modelOutput' : {0 : 'batch_size'}}
                 ) 
import onnx
from onnx_tf.backend import prepare
import onnxruntime
model_onnx = onnx.load('bilstm.onnx')
tf_rep = prepare(model_onnx)
tf_rep.export_graph('./tf_model')



/home/hydroxide/.local/lib/python3.7/site-packages/torch/onnx/utils.py:1189: UserWarning: Provided key modelInput for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
/home/hydroxide/.local/lib/python3.7/site-packages/torch/onnx/utils.py:1189: UserWarning: Provided key modelOutput for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
/home/hydroxide/.local/lib/python3.7/site-packages/torch/onnx/symbolic_opset9.py:2099: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  "or define the initial states (h0/c0) as inputs of the model. ")
/home/hydroxide/.local/lib/py

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/hydroxide/.local/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/hydroxide/.local/lib/python3.7/site-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/home/hydroxide/.local/lib/python3.7/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


2021-09-19 21:38:29.819600: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


/home/hydroxide/.local/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/hydroxide/.local/lib/python3.7/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/home/hydroxide/.local/lib/python3.7/site-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Assets written to: ./tf_model/assets


INFO:tensorflow:Assets written to: ./tf_model/assets


In [ ]:
import tensorflow_model_optimization as tfmot

quant_aware_model = tfmot.quantization.keras.quantize_model(tf_rep)
quant_aware_model.summary()

In [6]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("./tf_model")
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_enable_resource_variables = True

converter.experimental_new_converter =True
tflite_model = converter.convert()

# Save the model
with open("bilstm.tflite", 'wb') as f:
    f.write(tflite_model)

2021-09-19 21:38:36.705918: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-09-19 21:38:36.705948: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-09-19 21:38:36.705955: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored change_concat_input_ranges.
2021-09-19 21:38:36.706713: I tensorflow/cc/saved_model/reader.cc:38] Reading SavedModel from: ./tf_model
2021-09-19 21:38:36.733441: I tensorflow/cc/saved_model/reader.cc:90] Reading meta graph with tags { serve }
2021-09-19 21:38:36.733478: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: ./tf_model
2021-09-19 21:38:36.742094: I tensorflow/cc/saved_model/loader.cc:211] Restoring SavedModel bundle.
2021-09-19 21:38:36.776605: I tensorflow/cc/saved_model/loader.cc:195] Running initialization op on SavedModel bundle at path: ./tf_model
2021-09-19 21:38:36.7

In [ ]:
interpreter = tf.lite.Interpreter(model_path="bilstm.tflite")
interpreter.allocate_tensors()
    
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
output_details

In [ ]:
csv = pd.read_csv("data/training/speaker_1/20210824_1/61.csv")
columns = [x for x in list(csv.columns) if "Eye" not in x]
columns.remove("Timecode")
columns.remove("BlendShapeCount")
csv[columns].var().sort_values()
#df = preprocess_viseme("data/training/speaker_1/20210824_1/61.csv", pad_len_in_secs=pad_len_in_secs, 
#                                   resample_to=target_framerate, blendshapes=["MouthClose","MouthFunnel"])
#df.shape
#[df.iloc[0]["EyeLookInLeft"]]
    #csv[columns] = pd.np.digitize(csv[columns], np.linspace(0,1,11))
    
    #split = csv["Timecode"].str.split(':')
    #minute = split.str[1].astype(int)
    #second = split.str[2].astype(int)
    #frame = split.str[3].astype(float)
    #minute -= minute[0]
    #ms
    #step = minute * 60 + second
    #csv["step"] = step
    #return csv.drop_duplicates(["step"])[["step", "MouthClose","MouthFunnel","MouthPucker","JawOpen"]]
    
# if we want to use softmax across each blendshape as a one-hot
    #return np.reshape(vals, (vals.shape[0], vals.shape[1], 1))
    #one_hot = np.zeros((vals.shape[0], vals.shape[1], 11, 1))
    #oh = np.eye(11)
    #for row in range(vals.shape[0]):
    #    for t in range(vals.shape[1]):
    #        one_hot[row, :, :, 0] = np.eye(11)[int(vals[row,t])-1]
    #return one_hot